In [77]:
from collections import Counter
from nltk.corpus import wordnet
# from cluster_separation import cluster_separator_from_txt_file, read_cluster_file, create_clusters
import os
import re
import importlib


In [78]:
def create_clusters(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    clusters = []
    for line in lines:
        cluster = line.strip().replace('"', "").replace('[', "").replace(']', "").replace("'", "").split(',')
        if "Cluster" not in cluster[0]:
            cluster = [word.strip() for word in cluster] 
            clusters.append(cluster)

    return clusters

In [79]:
def write_labels_to_file(labels, path_to_file):
    with open(path_to_file, 'r') as f:
        lines = f.readlines()

    current_cluster = 0
    for i, line in enumerate(lines):
        if line.startswith("Cluster "):
            current_cluster += 1
            lines[i] = f"{line.strip()} ({labels[current_cluster-1]})\n"

    with open(path_to_file, 'w') as f:
        f.writelines(lines)

In [86]:
HOME_PATH = r"C:\Users\Nauel\Desktop\Lavoro UNI"
n_clusters = [5, 8, 10, 15, 20, 80, 100, 125, 150, 175, 200, 250, 300]
PATH_INPUT = os.path.join(HOME_PATH, "cluster_loop_clean", "definitions", f"Clusters con {n_cluster} gruppi & definizioni.txt")
PATH_OUTPUT = os.path.join(HOME_PATH, "cluster_loop_clean", "labels", f"Clusters con {n_cluster} gruppi & definizioni.txt")

clusters = create_clusters(PATH_INPUT)

# with open('most_common_hypernyms.txt', 'r') as file:
#     predefined_labels = [line.strip() for line in file]
#custom lables  
predefined_lables = ["family", "sport", "art", "manual labor", "intellectual labor/profession", "domestic work", "health work", "school", "free time/hobbies",  "leadership", "subordinate role", "parenthood", "home", "neutral"]

In [87]:
predefined_lables

['family',
 'sport',
 'art',
 'manual labor',
 'intellectual labor/profession',
 'domestic work',
 'health work',
 'school',
 'free time/hobbies',
 'leadership',
 'subordinate role',
 'parenthood',
 'home',
 'neutral']

In [85]:
import os

def read_clusters_from_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    clusters = []
    for line in lines:
        cluster = line.strip().replace('"', "").replace('[', "").replace(']', "").replace("'", "").split(',')
        if "Cluster" not in cluster[0]:
            cluster = [word.strip() for word in cluster] 
            clusters.append(cluster)

    return clusters

def write_labels_to_file(labels, input_path, output_path):
    with open(input_path, 'r') as f:
        lines = f.readlines()

    current_cluster = 0
    for i, line in enumerate(lines):
        if line.startswith("Cluster "):
            current_cluster += 1
            lines[i] = f"{line.strip()} ({labels[current_cluster-1]})\n"

    with open(output_path, 'w') as f:
        f.writelines(lines)

# Set the paths
HOME_PATH = r"C:\Users\Nauel\Desktop\Lavoro UNI"
n_clusters = [5, 8, 10, 15, 20, 80, 100, 125, 150, 175, 200, 250, 300]

for n_cluster in n_clusters:
    input_file = os.path.join(HOME_PATH, "cluster_loop_clean", "definitions", f"Clusters con {n_cluster} gruppi & definizioni.txt")
    output_file = os.path.join(HOME_PATH, "cluster_loop_clean", "labels_custom", f"Clusters con {n_cluster} gruppi & definizioni.txt")

    # Read clusters from the input file
    clusters = read_clusters_from_file(input_file)

    # Extract labels for the clusters
    cluster_labels = compute_cluster_labels(clusters, predefined_labels)

    # Write the labels to the output file
    write_labels_to_file(cluster_labels, input_file, output_file)


c:\Users\Nauel\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(
c:\Users\Nauel\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(
c:\Users\Nauel\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(
c:\Users\Nauel\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgradin

In [55]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def compute_cluster_labels(clusters, predefined_labels):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    
    # Convert clusters to embeddings
    cluster_texts = [' '.join(cluster) for cluster in clusters]
    cluster_embeddings = convert_clusters_to_embeddings(cluster_texts, model)
    
    # Convert predefined labels to embeddings
    predefined_labels_texts = [' '.join(label.split('_')) for label in predefined_labels]
    predefined_labels_embeddings = convert_clusters_to_embeddings(predefined_labels_texts, model)
    
    # Compute similarity scores
    similarity_scores = compute_similarity_scores(cluster_embeddings, predefined_labels_embeddings)
    
    # Find the most similar label for each cluster
    cluster_labels = [predefined_labels[np.argmax(scores)] for scores in similarity_scores]
    
    return cluster_labels

def convert_clusters_to_embeddings(cluster_texts, model):
    cluster_embeddings = model.encode(cluster_texts, convert_to_tensor=True)
    return cluster_embeddings

def compute_similarity_scores(cluster_embeddings, predefined_labels_embeddings):
    similarity_scores = cosine_similarity(cluster_embeddings, predefined_labels_embeddings)
    return similarity_scores

cluster_labels = compute_cluster_labels(clusters, predefined_labels)
print(cluster_labels)


c:\Users\Nauel\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


['businessperson', 'associate', 'skilled_worker', 'clergyman', 'associate', 'businessperson', 'skilled_worker', 'worker', 'professional', 'businessperson', 'performer', 'worker', 'merchant', 'intellectual', 'communicator', 'communicator', 'parent', 'medical_practitioner', 'communicator', 'employee', 'administrator', 'communicator', 'skilled_worker', 'sexual_intercourse', 'athlete', 'clergyman', 'worker', 'communicator', 'skilled_worker', 'creator', 'worker', 'skilled_worker', 'employee', 'scientist', 'sexual_intercourse', 'performer', 'worker', 'businessperson', 'worker', 'machine', 'medical_practitioner', 'individual', 'communicator', 'businessperson', 'parent', 'parent', 'educator', 'sexual_intercourse', 'artist', 'musician', 'clergyman', 'athlete', 'parent', 'clergyman', 'artist', 'medical_practitioner', 'intellectual', 'clergyman', 'craftsman', 'athlete', 'craftsman', 'artist', 'parent', 'communicator', 'communicator', 'worker', 'parent', 'athlete', 'communicator', 'businessperson'

In [19]:
for n_cluster in n_clusters:
    PATH_INPUT = os.path.join(HOME_PATH, "cluster_loop_clean", "definitions", f"Clusters con {n_cluster} gruppi & definizioni.txt")
    PATH_OUTPUT = os.path.join(HOME_PATH, "cluster_loop_clean", "labels", f"Clusters con {n_cluster} gruppi & definizioni.txt")
    clusters = create_clusters(PATH_INPUT)
    labels = [get_most_similar_labels(cluster, predefined_labels) for cluster in clusters]
    write_labels_to_file(labels, PATH_OUTPUT)


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

def create_clusters(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    clusters = []
    for line in lines:
        cluster = line.strip().replace('"', "").replace('[', "").replace(']', "").replace("'", "").split(',')
        if "Cluster" not in cluster[0]:
            cluster = [word.strip() for word in cluster]
            clusters.append(cluster)

    return clusters

def write_labels_to_file(labels, path_to_file):
    with open(path_to_file, 'r') as f:
        lines = f.readlines()

    current_cluster = 0
    for i, line in enumerate(lines):
        if line.startswith("Cluster "):
            current_cluster += 1
            lines[i] = f"{line.strip()} ({labels[current_cluster-1]})\n"

    with open(path_to_file, 'w') as f:
        f.writelines(lines)

def associate_clusters_with_labels(clusters, predefined_labels):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    def calculate_similarity(sentence1, sentence2):
        inputs = tokenizer.encode_plus(sentence1, sentence2, add_special_tokens=True, return_tensors='pt')
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            sentence_embeddings = torch.mean(outputs.last_hidden_state, dim=1)
        similarity = 1 - cosine(sentence_embeddings[0].cpu(), torch.mean(sentence_embeddings, dim=0).cpu())
        return similarity



    cluster_labels = []
    for cluster in clusters:
        max_similarity = 0
        cluster_label = None
        for label in predefined_labels:
            similarity = calculate_similarity(' '.join(cluster), label)
            if similarity > max_similarity:
                max_similarity = similarity
                cluster_label = label
        cluster_labels.append(cluster_label)

    return cluster_labels

HOME_PATH = r"C:\Users\Nauel\Desktop\Lavoro UNI"
n_clusters = [5, 8, 10, 15, 20, 80, 100, 125, 150, 175, 200, 250, 300]

for n_cluster in n_clusters:
    PATH_INPUT = os.path.join(HOME_PATH, "cluster_loop_clean", "definitions", f"Clusters con {n_cluster} gruppi & definizioni.txt")
    PATH_OUTPUT = os.path.join(HOME_PATH, "cluster_loop_clean", "labels", f"Clusters con {n_cluster} gruppi & definizioni.txt")
    clusters = create_clusters(PATH_INPUT)
    labels = associate_clusters_with_labels(clusters, predefined_labels)
    write_labels_to_file(labels, PATH_OUTPUT)